Заполните данные о необходимых переменных:

In [1]:
Today = '29/02/2024'
Mounth = 'Feb'

In [2]:
import os
from datetime import date
import pandas as pd
import numpy as np

In [3]:
Output_SUEK = "_".join([str(date.today()), 'SUEK', 'SAP', 'ARAP', f'{Mounth}.xlsx'])
Output_EURO = "_".join([str(date.today()), 'EUROCHEM', 'SAP', 'ARAP', f'{Mounth}.xlsx'])
Sheet_in_output_file = 'ARAP'

In [4]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column

Открываю следующие файлы:

In [7]:
query = f'''select 
    "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding","currencyCode", "sourceSystem",
CASE
        WHEN "bscCode" in ('130','135','BSC130','BSC135') 
        THEN 'Trade receivables'
        WHEN "bscCode" in ('160','165','BSC160','BSC165') 
        THEN 'Advances to suppliers'
        WHEN "bscCode" in ('170','175','BSC170','BSC175')
        THEN 'Capital advances'
        WHEN "bscCode" in ('140','145','BSC140','BSC145') 
        THEN 'Other receivables'
        WHEN "bscCode" in ('330','BSC330')
        THEN 'Provisions'
        WHEN "bscCode" in ('300','305','BSC300','BSC305') 
        THEN 'Trade payables'
        WHEN "bscCode" in ('310','315','BSC310','BSC315') 
        THEN 'Other payables'
        WHEN "bscCode" in ('360','365','BSC360','BSC365') 
        THEN 'Advances received'
        WHEN "bscCode" in ('350','355','BSC350','BSC355') 
        THEN 'Loans received'
        WHEN "bscCode" in ('210','215','BSC210','BSC215') 
        THEN 'Loans issued'
        WHEN "bscCode" in ('340','BSC340') 
        THEN 'Tax'
        WHEN "bscCode" in ('410','BSC410') 
        THEN 'Pension'
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    ("sourceSystem" = 'OEBS12_EU' or "sourceSystem" = 'ETL document loader') and
    "rowType" = 'M' and
    "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')  
union all select
     "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding","currencyCode", "sourceSystem",
CASE
        WHEN ("glAccount" like '62%' or "glAccount" IN (7699200310, 7699100210,4601000000, 4611000000, 1501010009, 1501010019, 1501010109, 1501010119, 2201010000,
        2201010008, 2201010009, 2201010010, 2201010018, 2201010019, 2201011000, 2201011008, 2201011009, 2201011010, 2201011018, 2201011019, 2201010098, 2201010099,
        2201020000, 2201020008, 2201030000, 2201030008, 2201010090, 2201010190, 2110000000, 2110000900, 2130000000, 2130000900, 2151000000, 2151000900, 2170000000,
        2170000900, 1210000000, 1210000900, 1260000000, 2181000000, 2181000900))
        THEN 'Trade receivables'
        WHEN "glAccount" IN (6002770900, 6002090900, 6002020000, 6002020099, 6002080000, 6002090000, 6002050000, 6002020900, 6002070000, 6002070099, 6002770000,
        6002050900, 6002070900, 6099000215, 6099000224, 6099000212, 6099000217, 6099000219, 6099000619, 6099000120, 6099000123, 6099000820, 6099000430, 6099000460,
        6099000470, 6099000490, 7699100410, 6099000830, 6099000810, 6002030000, 6002040000, 6002030900, 6002040900, 6099000440, 6099000450, 6099000203, 6099000615, 
        6002140000, 6002140099, 6099000616, 6002150000, 7610001900, 7610001000, 7610001099, 2301010100, 2301010108, 2301010110, 2301010118, 2301010200, 2301010208,
        2301010210, 2301010218, 2301010300, 2301010308, 2301010400, 2301010408, 2301011100, 2301011108, 2301011110, 2301011118, 2301011200, 2301011208, 2301011210,
        2301011218, 2301011300, 2301011308, 2302010000, 2302010007, 1611000000, 1613000000, 1615000000, 7610007900, 7610007000, 7610005000, 7699100620, 7699100420)
        THEN 'Advances to suppliers'
        WHEN "glAccount" IN (6002010000, 6002010099, 6002010900, 6099000612, 6099000211, 2301020100, 2301020108, 2301020110, 2301020118, 2301020200, 2301020208,
        2301020210, 2301020218, 2301020300, 2301020308, 2301020310, 2301020318, 2301021100, 2301021108, 2301021110, 2301021118, 2301021200, 2301021208, 2301021210,
        2301021218, 2301021300, 2301021308, 2301021310, 2301021318, 2301000008, 2301001008, 2302020000, 2302020007, 1612000000, 1614000000)
        THEN 'Capital advances'
        WHEN "glAccount" IN (6001089000, 7625003000, 7610003000, 7615003000, 7699100550, 6099000710, 7699200500, 7615004000, 7629009000, 7625013000, 7625013900,
        7699100200, 7629000900, 7610406000, 7610406900, 7610405000, 7610405900, 7630707000, 7622020000, 7620801000, 7620801900, 7630705000, 7629000099, 7629000000,
        7610004000, 7618000900, 7618000000, 6001080900, 6001080000, 7622010000, 7620703000, 7620704000, 7620701000, 7620701900, 7620702000, 7622030000, 7622040000,
        7699900000, 7610500000, 7610500099, 7699100240, 7630702000, 7630703000, 7630706000, 7625006000, 7625007000, 7625001000, 7625001900, 7620200000, 7620200900,
        2601030000, 2601030008, 2601040000, 2601040008, 2601050009, 2602000000, 2602000008, 2602000009, 2602000010, 2602000018, 2602000019, 2602001000, 2602001008,
        2602001010, 2602001018, 2602010000, 2602010009, 2602060100, 2602000090, 2603000000, 2603000100, 36307060, 1284000000, 1284000900, 1630000000, 1282000000, 
        1282000900, 1251000000, 2601010009, 2601020009)
        THEN 'Other receivables'
        WHEN "glAccount" IN (2201910000, 1290000000, 2190000000, 2301910100, 2602900000)
        THEN 'Provisions'
        WHEN ("glAccount" IN (1022200000))
        THEN 'FX Spot'
        WHEN ("glAccount" IN (1110009000, 1270000000, 1271000000, 2204000000, 2604010100, 2604020100, 2604021110, 2604021200, 7621156000))
        THEN 'Loans issued'
        WHEN ("glAccount" IN (1410000000, 1430010000, 1430030000, 1430050000, 1430060000, 1430070000))
        THEN 'Tax'
        WHEN ("glAccount" IN (2602000020))
        THEN 'MTM Derivatives'        
        WHEN ("glAccount" IN (7610300000))
        THEN 'Dividends'        
        WHEN ("glAccount" IN (7611102000))
        THEN 'Capital expenses'       
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    ("sourceSystem" = 'SAP ERP' or "sourceSystem" = 'SAP ECP') and
    "rowType" = 'M' and
    "reportType" = 'AR' and 
    "reportDate" = TO_DATE('{Today}', 'DD/MM/YY') 
union all select
 "customerName","internalExternalEN","orgName", "companyCode", "dueDate", "presentAmountUSD", "dealNumber", "holding", "currencyCode", "sourceSystem",
CASE
        WHEN "glAccount" like '63%'
        THEN 'Provisions'
        WHEN ("glAccount" IN (5201010100, 5201010110, 5201010200, 5201011100, 5201011110, 5201011200))
        THEN 'Trade payables'
        WHEN ("glAccount" IN (5501000000, 5501010000, 5502000000, 5503020000, 5503030000, 5503030010, 5503010000))
        THEN 'Other payables'
        WHEN ("glAccount" like '62%' or "glAccount" IN (5301010000, 5301010010, 7620002000, 0035100000))
        THEN 'Advances received'
        WHEN ("glAccount" like '60%' or "glAccount" IN (5101010100, 5101020110, 5101021110, 5104040000, 5109000000, 5503050000, 5505010200, 5505010210, 5505011210, 5505020100, 
        7610101000, 7610402000, 7610404000, 7610405000, 7610601000, 7610603000, 7610691000, 7619000000, 7620002000, 0033970000, 0033500000, 0033600000, 0033960000, 0033140000,
        0033130000, 0033120000, 0033110000, 0033800000, 0033950000, 0033970100))
        THEN 'Other payables'
        WHEN ("glAccount" IN (0032200000, 0032310000))
        THEN 'Pension'
        WHEN ("glAccount" IN (0032300000, 0032150000, 0032140000, 0032130000, 0032120000, 0032110000, 0031500000, 0033970200))
        THEN 'Social'
        WHEN ("glAccount" IN (0031940000, 0031930000, 0031920000, 0031910000, 0031800000, 0031700000, 0031600000, 0031330000, 0031320000, 0031220000, 0031210000, 0031100000))
        THEN 'Tax'
        WHEN ("glAccount" IN (0030200000))
        THEN 'Loans received'
        ELSE 'Unknown'
    END AS "accountPurpose"
from RISKACCESS."unifiedARReport"
where 
    ("sourceSystem" = 'SAP ERP' or "sourceSystem" = 'SAP ECP') and
    "rowType" = 'M' and
    "reportType" = 'AP' and 
    "reportDate" = TO_DATE('{Today}', 'DD/MM/YY') 
    '''

In [8]:
data_export = export_from_RISKCUSTOM(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [10]:
data_work = data_export.query('holding != "EXTERNAL"').reset_index(drop=True)
# фильтрация по перемнной Group

data_work = data_work[~(data_work.orgName.isna() & data_work.companyCode.isna())].reset_index(drop=True)
data_work['org_code'] = merge_Mapping(data_work, col='orgName')
data_work['CompCode_concat'] = data_work['companyCode'].fillna('')
data_work.loc[data_work.CompCode_concat == '','CompCode_concat'] = data_work.loc[data_work.CompCode_concat == '','org_code']
data_work['CompMap'] = merge_SalesUnits(data_work, col='CompCode_concat', merge_col='ocpSegment')

data_work['CtyCode'] = merge_Mapping(data_work, col='customerName').fillna('External')
data_work['CtyMap'] = merge_SalesUnits(data_work, col='CtyCode', merge_col='ocpSegment').fillna('External')

data_work['AR_flag'] = data_work['accountPurpose']\
                                            .isin(['Trade receivables', 'Other receivables', 'Provisions'])\
                                            .astype(int)
data_work['AP_flag'] = data_work['accountPurpose']\
                                            .isin(['Trade payables', 'Other payables'])\
                                            .astype(int)

data_work['Amount_USD'] = data_work['presentAmountUSD'].replace({np.NaN : 0})
data_work['Amount_AR_USD'] = data_work['Amount_USD'] * data_work['AR_flag']
data_work['Amount_AP_USD'] = data_work['Amount_USD'] * data_work['AP_flag']

data_work = Period(data_work, day_for_count=Today, col_with_date='dueDate')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:44: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  day = pd.to_datetime(day_for_count)


Проверка на merge:

In [12]:
check_data = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CtyMap'] == 'External'),\
                                                    ['customerName', 'CtyCode', 'CtyMap']].drop_duplicates()
# check_data
# # check_data.to_excel('manual_map_ARAP_ECH.xlsx', index=False)

check_data2 = data_work.loc[(data_work['internalExternalEN'] == 'Internal') & (data_work['CompMap'] == 'External'),\
                                                    ['orgName', 'CompCode_concat','CompMap']].drop_duplicates()
check_data2
# check_data

,orgName,CompCode_concat,CompMap
221490,"АО ""СГК - Алтай""",External,External
274674,"АО ""СГК - Новосибирск""",External,External


In [40]:
BS_data = data_work[(data_work.accountPurpose.str.contains('Trade')) | (data_work.accountPurpose.str.contains('Other'))].reset_index(drop=True)
BS_data['Comp_hold'] = merge_SalesUnits(BS_data, col='CompCode_concat', merge_col='holding')
BS_data['Cpty_hold'] = merge_SalesUnits(BS_data, col='CtyCode', merge_col='holding')

for Group in ['SUEK', 'EUROCHEM']:
    BS_data_group = BS_data[(BS_data.Comp_hold == Group) & (BS_data.Cpty_hold == Group)]
    BS_data_group = BS_data_group[BS_data_group.CompMap != 'External']
    BS_data_group = BS_data_group[['CompMap', 'CtyMap', 'Amount_USD']]
    BS_pivot = pd.pivot_table(BS_data_group, values='Amount_USD', index='CompMap', aggfunc='sum', columns='CtyMap',)
    
    if Group == 'EUROCHEM':
        BS_pivot_EURO = BS_pivot
    if Group == 'SUEK':
        BS_pivot_SUEK = BS_pivot

CtyMap,NTC,RUFERT,SGC,SUEK INT,SUEK RU
CompMap,,,,,
NTC,-854523.38,-788860.66,104028.71,5.206835e+06,-3.138018e+07
RUFERT,762525.26,0.03,-41841.61,NaN,-3.668486e+04
SGC,24726.75,41841.59,2810955.22,NaN,-9.478726e+07
SUEK INT,-5994313.06,NaN,NaN,1.306585e+06,-7.104478e+08
SUEK RU,5563925.66,42174.50,98281265.66,1.031097e+09,9.684607e+06


In [9]:
# assert 1==2

Запись в output файлы:

In [39]:
data_print_SUEK = data_work.query('holding == "SUEK"')[['customerName', 'CompCode_concat', 'currencyCode', 'sourceSystem',\
                    'accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','Amount_USD',\
                    'AR_flag', 'Amount_AR_USD', 'AP_flag', 'Amount_AP_USD', 'Days', 'Period']]
data_print_EURO = data_work.query('holding == "EUROCHEM"')[['customerName', 'CompCode_concat', 'currencyCode', 'sourceSystem',\
                    'accountPurpose', 'CompMap', 'CtyCode', 'CtyMap','Amount_USD',\
                    'AR_flag', 'Amount_AR_USD', 'AP_flag', 'Amount_AP_USD', 'Days', 'Period']]

In [25]:
BS_pivot_EURO.to_excel(Output_EURO, sheet_name='BS_to_BS')
new_list(data_print_EURO, Output_EURO, Sheet_in_output_file)

BS_pivot_SUEK.to_excel(Output_SUEK, sheet_name='BS_to_BS')
new_list(data_print_SUEK, Output_SUEK, Sheet_in_output_file)

Поиск встречных требований:

In [16]:
data_work.shape
data_work.columns

Index(['customerName', 'internalExternalEN', 'orgName', 'companyCode',
       'dueDate', 'presentAmountUSD', 'dealNumber', 'holding', 'currencyCode',
       'sourceSystem', 'accountPurpose', 'org_code', 'CompCode_concat',
       'CompMap', 'CtyCode', 'CtyMap', 'AR_flag', 'AP_flag', 'Amount_USD',
       'Amount_AR_USD', 'Amount_AP_USD', 'Days', 'Period'],
      dtype='object')

In [23]:
data_work.head(2)

,customerName,internalExternalEN,orgName,companyCode,dueDate,presentAmountUSD,dealNumber,holding,currencyCode,sourceSystem,...,CompMap,CtyCode,CtyMap,AR_flag,AP_flag,Amount_USD,Amount_AR_USD,Amount_AP_USD,Days,Period
0,IVORY CHOICE TERMINAL CO,External,USBFC: EuroChem North America Corporation,USBFC,NaT,-1164.00,None,EUROCHEM,USD,OEBS12_EU,...,NAMSALES,External,External,0,1,-1164.00,-0.0,-1164.0,0.0,<1M
1,NUTRAGRO,External,USBFC: EuroChem North America Corporation,USBFC,NaT,-3757.94,None,EUROCHEM,USD,OEBS12_EU,...,NAMSALES,External,External,0,0,-3757.94,-0.0,-0.0,0.0,<1M


In [27]:
index_datawork_to_print = []

# Фильт по Counterparty_Group - только внутригрупповые
data_E_to_C = data_work.query('CtyMap != "External"')
data_E_to_C['Index_copy'] = data_E_to_C.index

# Кокатенация значений
data_E_to_C[['dealNumber',  'dueDate']] = data_E_to_C[['dealNumber',  'dueDate']].fillna('_')
data_E_to_C['concat_col_EC'] = data_E_to_C[['CompCode_concat', 'CtyCode', 'dealNumber',  'dueDate', 'currencyCode', 'Amount_USD']].astype(str).agg('.'.join, axis=1)
data_E_to_C['concat_col_CE'] = data_E_to_C[['CtyCode', 'CompCode_concat', 'dealNumber',  'dueDate', 'currencyCode', 'Amount_USD']].astype(str).agg('.'.join, axis=1)

# Данные с повторяющимися значениями
doubles_data = data_E_to_C[data_E_to_C.concat_col_EC.isin(data_E_to_C.concat_col_CE)]
doubles_data = doubles_data.sort_values(by=['dealNumber',  'dueDate', 'currencyCode', 'Amount_USD'])
# Проверка на отсутсвие необходимости суммировать значения amoutn_outstanding
group_data = data_E_to_C.pivot_table(index=['CompCode_concat', 'CtyCode', 'dealNumber',  'dueDate', 'currencyCode', 'Amount_USD'], values='Index_copy', aggfunc={'Index_copy': list}).reset_index()
group_data['Index_count'] = group_data['Index_copy'].apply(len)
assert group_data.Index_count.unique().tolist() == [1]
# Проверка наличия пары у кадого значения
group_data = doubles_data.pivot_table(index=['dealNumber',  'dueDate', 'currencyCode', 'Amount_USD'], values='Index_copy', aggfunc={'Index_copy': list}).reset_index()
group_data['Index_count'] = group_data['Index_copy'].apply(len)
assert group_data.Index_count.unique().tolist() == [2]

# В список index_dataDebt_to_print добавлена только одна запись из равной пары
group_data['index_to_print'] = group_data['Index_copy'].str[0]
index_to_print_list = group_data.index_to_print.tolist()
index_datawork_to_print += index_to_print_list
# Проверка, что взята ровно половина повторяющихся данных
assert len(index_datawork_to_print) == len(doubles_data)/2

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_7136\2659141684.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_E_to_C['Index_copy'] = data_E_to_C.index
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_7136\2659141684.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_E_to_C[['dealNumber',  'dueDate']] = data_E_to_C[['dealNumber',  'dueDate']].fillna('_')
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_7136\2659141684.py:9: SettingWithCopyWarning: 
A value is trying to be set on a cop

AssertionError: 

In [35]:
ls = list(np.concatenate(group_data[group_data.Index_count >1].Index_copy.tolist()))

In [37]:
data_work.loc[[499268, 499544], :]
data_work.loc[ls, :].to_excel('2024_03_29_Douples_ARAP.xlsx')